#Analysis

In reviewing the data it looks like school size and spending per student are big factors for passing scores in reading and math. 15 schools brought back an average score of 65% which would imply all the schools should be in that range. I've found that the large size schools and schools that spend more than $630 per student scores start significantly falling compared to other schools. With that data I can assume that some schools poor performance brought down the good schools performances. Charter schools did better in every category vs district schools. Medium and small size schools pass both math and reading at a much higher percentage that large size schools. 


Schools that spend more money % of passing both categories decreases as the bins increase. {<585 90%} {$645-680 53%}

Schools that spend <$585 per student scores are higher over all metrics {As bins change <585 scores stay at the top}

Schools that spend >$630 have an averge score of 58%

Schools that spend $630 or less have an average score of 86%

Avg Math Score	Avg Read Score	% Pass Math	% Pass Read	% Passed Both
Spending Ranges (Per Student)					
<$585	83.455399	83.933814	93.460096	96.610877	90.369459
$585-630	81.899826	83.155286	87.133538	92.718205	81.418596
$630-645	78.518855	81.624473	73.484209	84.391793	62.857656
$645-680	76.997210	81.027843	66.164813	81.133951	53.526855


In [143]:
import pandas as pd
from pathlib import Path

school_data_to_load = Path("schools_complete.csv")
student_data_to_load = Path("students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

In [2]:
df = pd.DataFrame(school_data_complete)

In [3]:
# Calculate the total number of unique schools
school_count = df['school_name'].nunique()
school_count

15

In [4]:
# Calculate the total number of students
student_count = df['student_name'].count()
student_count

39170

In [5]:
# Calculate the total budget
total_budget =  school_data['budget'].sum()
total_budget

24649428

In [7]:
# Calculate the average (mean) math score
average_math_score = df['math_score'].mean()
average_math_score

78.98537145774827

In [6]:
# Calculate the average (mean) reading score
average_reading_score = df['reading_score'].mean() 
average_reading_score

81.87784018381414

In [8]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = df[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [9]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = df[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [10]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = df[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [11]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({"Total Schools": school_count,"Total Students": student_count, "Total Budget": total_budget, "Avg Math Score": average_math_score, "Avg Read Score": average_reading_score, "% Math Passed": passing_math_percentage, "% Reading Passed": passing_reading_percentage, "% Overall Passed": overall_passing_rate,}, index=[0])


# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Read Score,% Math Passed,% Reading Passed,% Overall Passed
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [95]:
#Calculate the total student count
grouped_by_schooldf = df.groupby('school_name')

per_school_counts = grouped_by_schooldf['student_name'].count()

# Use the code provided to select the school type
school_types = school_data.set_index(["school_name"])["type"]


#grouped_by_schooldf.head()
#per_school_counts.head()

In [96]:
# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

In [97]:
# Calculate the average test scores
per_school_math = df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"]

#per_school_reading.head()

In [98]:
# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)].groupby(['school_name'])['student_name'].count() /per_school_counts *100
#school_passing_reading

In [99]:
school_passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)].groupby(['school_name'])['student_name'].count() /per_school_counts *100
#school_passing_math

In [100]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
].groupby(['school_name'])['student_name'].count() /per_school_counts *100
#passing_math_and_reading.head()

In [106]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
df1 = pd.DataFrame({'School Type' : school_types,
                    'Total Students' : per_school_counts,
                    'Total Budget' : per_school_budget,
                    'Student Budget' : per_school_capita,
                    'Average Math Score' : per_school_math,
                    'Average Reading Score' : per_school_reading,
                    '% Pass Reading' : school_passing_reading,
                    '% Pass Math' : school_passing_math,
                    '% Passed Both' : passing_math_and_reading})
df1
Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
#per_school_summary

,School Type,Total Students,Total Budget,Student Budget,Average Math Score,Average Reading Score,% Pass Reading,% Pass Math,% Passed Both
school_name,,,,,,,,,
Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,81.933280,66.680064,54.642283
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,97.039828,94.133477,91.334769
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,80.739234,65.988471,53.204476
Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,79.299014,68.309602,54.289887
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,97.138965,93.392371,90.599455
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,80.862999,66.752967,53.527508
Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,96.252927,92.505855,89.227166
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,81.316421,65.683922,53.513884
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,81.222432,66.057551,53.539172


In [145]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
df2 = df1.sort_values(by = '% Passed Both', ascending=False)
df2.head(5)

,School Type,Total Students,Total Budget,Student Budget,Average Math Score,Average Reading Score,% Pass Reading,% Pass Math,% Passed Both,School Size
school_name,,,,,,,,,,
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,97.039828,94.133477,91.334769,Medium (1000-2000)
Thomas High School,Charter,1635,1043130.0,638.0,83.418349,83.848930,97.308869,93.272171,90.948012,Medium (1000-2000)
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,97.138965,93.392371,90.599455,Medium (1000-2000)
Wilson High School,Charter,2283,1319574.0,578.0,83.274201,83.989488,96.539641,93.867718,90.582567,Large (2000-5000)
Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,95.945946,94.594595,90.540541,Small (<1000)


In [144]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
df3 = df1.sort_values(by = '% Passed Both')
df3.head(5)

,School Type,Total Students,Total Budget,Student Budget,Average Math Score,Average Reading Score,% Pass Reading,% Pass Math,% Passed Both,School Size
school_name,,,,,,,,,,
Rodriguez High School,District,3999,2547363.0,637.0,76.842711,80.744686,80.220055,66.366592,52.988247,Large (2000-5000)
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,80.739234,65.988471,53.204476,Large (2000-5000)
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,81.316421,65.683922,53.513884,Large (2000-5000)
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,80.862999,66.752967,53.527508,Large (2000-5000)
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,81.222432,66.057551,53.539172,Large (2000-5000)


In [114]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(['school_name'])['math_score'].mean()
tenth_graders_scores = tenth_graders.groupby(['school_name'])['math_score'].mean()
eleventh_graders_scores = eleventh_graders.groupby(['school_name'])['math_score'].mean()
twelfth_graders_scores = twelfth_graders.groupby(['school_name'])['math_score'].mean()


# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({'9th' : ninth_graders_scores,
                    '10th' : tenth_graders_scores,
                    '11th' : eleventh_graders_scores,
                    '12th' : twelfth_graders_scores,
                    })

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade



,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


In [113]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_readscores = ninth_graders.groupby(['school_name'])['reading_score'].mean()
tenth_graders_readscores = tenth_graders.groupby(['school_name'])['reading_score'].mean()
eleventh_graders_readscores = eleventh_graders.groupby(['school_name'])['reading_score'].mean()
twelfth_graders_readscores = twelfth_graders.groupby(['school_name'])['reading_score'].mean()


# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
read_scores_by_grade = pd.DataFrame({'9th' : ninth_graders_readscores,
                    '10th' : tenth_graders_readscores,
                    '11th' : eleventh_graders_readscores,
                    '12th' : twelfth_graders_readscores,
                    })

# Minor data wrangling
read_scores_by_grade.index.name = None

# Display the DataFrame
read_scores_by_grade


,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


In [115]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [119]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = df1.copy()

In [123]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df['Student Budget'] , spending_bins, labels=labels)
school_spending_df.head()

,School Type,Total Students,Total Budget,Student Budget,Average Math Score,Average Reading Score,% Pass Reading,% Pass Math,% Passed Both,Spending Ranges (Per Student)
school_name,,,,,,,,,,
Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,81.933280,66.680064,54.642283,$585-630
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,97.039828,94.133477,91.334769,<$585
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,80.739234,65.988471,53.204476,$630-645
Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,79.299014,68.309602,54.289887,$630-645
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,97.138965,93.392371,90.599455,$585-630


In [124]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Pass Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Pass Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passed Both"]

In [125]:
df5 = pd.DataFrame({'Avg Math Score' : spending_math_scores,
                    'Avg Read Score' : spending_reading_scores,
                    '% Pass Math' : spending_passing_math,
                    '% Pass Read' : spending_passing_reading,
                    '% Passed Both' : overall_passing_spending 
                    })
df5

,Avg Math Score,Avg Read Score,% Pass Math,% Pass Read,% Passed Both
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


In [126]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [128]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

df1["School Size"] = pd.cut(school_spending_df['Total Students'] , size_bins, labels=labels)
                      

In [133]:
# Calculate averages for the desired columns. 
size_math_scores = df1.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = df1.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = df1.groupby(["School Size"]).mean()["% Pass Math"]
size_passing_reading = df1.groupby(["School Size"]).mean()["% Pass Reading"]
size_overall_passing = df1.groupby(["School Size"]).mean()["% Passed Both"]

In [135]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
df6 = pd.DataFrame({'Avg Math Score' : size_math_scores,
                    'Avg Read Score' : size_reading_scores,
                    '% Pass Math' : size_passing_math,
                    '% Pass Read' : size_passing_reading,
                    '% Passed Both' : size_overall_passing
                    })

# Display results
df6

,Avg Math Score,Avg Read Score,% Pass Math,% Pass Read,% Passed Both
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [140]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = df1.groupby(['School Type'])['Average Math Score'].mean()
type_reading_scores = df1.groupby(['School Type'])['Average Reading Score'].mean()
type_passing_math = df1.groupby(['School Type'])['% Pass Math'].mean()
type_passing_reading = df1.groupby(['School Type'])['% Pass Reading'].mean()
type_overall_passing = df1.groupby(['School Type'])['% Passed Both'].mean()



In [141]:
df7 = pd.DataFrame({'Avg Math Score' : type_math_scores,
                    'Avg Read Score' : type_reading_scores,
                    '% Pass Math' : type_passing_math,
                    '% Pass Read' : type_passing_reading,
                    '% Passed Both' : type_overall_passing
                    })

df7

,Avg Math Score,Avg Read Score,% Pass Math,% Pass Read,% Passed Both
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
